In [ ]:
import pandas as pd
import numpy as np

import featuretools as ft
from datetime import datetime
from uuid import uuid4
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
import xgboost as xgb

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

In [ ]:
# https://github.com/IBM/telco-customer-churn-on-icp4d
URL_TO_DATA = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"

CUSTOMERS = [
    # "customerID",
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "ContractStartDate",
]
SUBSCRIPTIONS = [
    # "customerID",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]

BILLING = [
    # "customerID",
    "tenure",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "MonthlyCharges",
    "TotalCharges",
    "Churn",
]

BACK_COUNT_DATE = datetime.fromisoformat("2022-01-01")
TEST_SIZE = 0.2
VALID_SIZE = 0.25
RANDOM_STATE = 42

- ft titanic https://www.kaggle.com/code/liananapalkova/automated-feature-engineering-for-titanic-dataset/notebook
- the dataset https://github.com/IBM/telco-customer-churn-on-icp4d/blob/master/data/Telco-Customer-Churn.csv
- titanic https://medium.com/dataexplorations/tool-review-can-featuretools-simplify-the-process-of-feature-engineering-5d165100b0c3
- time indexing recomandations https://stackoverflow.com/questions/49711987/how-do-i-prevent-data-leakage-with-featuretools




### Denormalize into 
- customer_df:      customer_id (PK), subscription_id (FK), gender, SeniorCitizen, Partner, Dependents, tenure, Churn
- services_df:      service_id (PK), service_name
- subscription_df:  subscription_id (PK), service_id, customer_id
- billing_df:       billing_id, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges


In [ ]:
# in case of CERTIFICATE_VERIFY_FAILED run Install Certificates.command
# see also https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org
df = pd.read_csv(filepath_or_buffer=URL_TO_DATA, index_col=0)

In [ ]:
# convert
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# generate syntatic time index
df["ContractStartDate"] = list(
    map(
        lambda tenure, dat=BACK_COUNT_DATE: dat - relativedelta(months=-tenure),
        df["tenure"],
    )
)
# df["customerID"] = df.index
df["billingId"] = [str(uuid4()) for _ in range(df.shape[0])]
df["subscriptionId"] = [str(uuid4()) for _ in range(df.shape[0])]

# convert to 1/0
df["Churn"] = np.where(df["Churn"] == 'Yes', 1, 0)

In [ ]:
y = df["Churn"]
X = df.drop(columns=["Churn"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=VALID_SIZE, random_state=RANDOM_STATE
)  # 0.25 x 0.8 = 0.2

In [ ]:
# df_customers = df[CUSTOMERS].copy(deep=True)
# df_subscriptions = df[SUBSCRIPTIONS].copy(deep=True)
# df_billing = df[BILLING].copy(deep=True)

In [ ]:
# dataframes = {
#     "customers": (df_customers, "customer_id"),
#     "subscriptions": (df_subscriptions, "customer_id", "customer_id"),
#     "billing": (df_billing, "customer_id", "customer_id"),
# }


# relationships = [
#     ("customers", "customer_id", "subscriptions", "customer_id"),
#     ("subscriptions", "customer_id", "billing", "customer_id"),
# ]

# feature_matrix_customers, features_defs = ft.dfs(
#     dataframes=dataframes,
#     relationships=relationships,
#     target_dataframe_name="customers",
# )
# feature_matrix_customers

In [ ]:
# https://featuretools.alteryx.com/en/stable/generated/featuretools.EntitySet.add_dataframe.html#featuretools.EntitySet.add_dataframe
# entity_set = ft.EntitySet("customer_churn_entities")

# entity_set.add_dataframe(
#     dataframe_name="customers",
#     index="customerID",
#     time_index="ContractStartDate",
#     dataframe=df_customers,
# )

# entity_set.add_dataframe(
#     dataframe_name="subscriptions",
#     index="customerID",
#     dataframe=df_subscriptions,
# )

# entity_set.add_dataframe(
#     dataframe_name="billing",
#     index="customerID",
#     dataframe=df_billing,
# )

# relationships = [
#     ("customers", "customerID", "subscriptions", "customerID"),
#     ("subscriptions", "customerID", "billing", "customerID"),
# ]

# entity_set.add_relationships(relationships=relationships)

In [ ]:
es = ft.EntitySet(id="customer_churn")

es.add_dataframe(
    dataframe_name="customers",
    index="customerID",
    time_index="ContractStartDate",
    dataframe=X_train,
)

es.normalize_dataframe(
    base_dataframe_name="customers",
    new_dataframe_name="subscriptions",
    index="subscriptionId",
)

es.normalize_dataframe(
    base_dataframe_name="customers", new_dataframe_name="billings", index="billingId"
)

In [ ]:
es.plot()

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es, target_dataframe_name="customers", max_depth=3
)




In [ ]:
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_defs)
feature_matrix_enc

In [ ]:
features_enc

In [ ]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_test, y)

#y_pred = xgb_model.predict(X)

#print(confusion_matrix(y, y_pred))

In [ ]:
np.where(df["Churn"] == 'Yes', 1, 0)